In [1]:
source("yaara7.r")


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Registered S3 method overwritten by 'ggtree':
  method      from 
  identify.gg ggfun

ggtree v3.2.1  For help: https://yulab-smu.top/treedata-book/

If you use ggtree in published research, please cite the most appropriate paper(s):

1. Guangchuang Yu. Using ggtree to visualize data on tree-like structures. Current Protocols in Bioinformatics. 2020, 69:e96. doi:10.1002/cpbi.96
2. Guangchuang Yu, Tommy Tsan-Yuk Lam, Huachen Zhu, Yi Guan. Two methods for mapping and visualizing associated data on phylogeny using ggtree. Molecular Biology and Evolution. 2018, 35(12):3041-3043. doi:10.1093/molbev/msy194
3. Guangchuang Yu, David Smith, Huachen Zhu, Yi Guan, Tommy Tsan-Yuk Lam. ggtree: an R package for visualization and annotation of phylogenetic trees with their covariates and other associate

In [2]:
library(data.table)


Attaching package: 'data.table'


The following objects are masked from 'package:dplyr':

    between, first, last




In [3]:
master = fread("Masterlist lineage tree_MergeMelanoma-MC-NSCLC_v9_WES_20230502ZC.csv")

In [4]:
annot = fread("annot.csv")

In [5]:
martin = fread("all_copy.csv")

In [6]:
martin$V1 = annot$Chr

In [7]:
library(tidyverse)

-- Attaching packages --------------------------------------- tidyverse 1.3.1 --

v tibble  3.1.8     v purrr   1.0.1
v tidyr   1.2.0     v forcats 0.5.1
v readr   2.1.2     

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x data.table::between() masks dplyr::between()
x tidyr::expand()       masks ggtree::expand()
x dplyr::filter()       masks stats::filter()
x data.table::first()   masks dplyr::first()
x dplyr::lag()          masks stats::lag()
x data.table::last()    masks dplyr::last()
x purrr::transpose()    masks data.table::transpose()



In [8]:
long_martin = martin %>% pivot_longer(cols = !V1, values_to = "cn") %>% rename(chrom=V1)

In [9]:
long_martin$cnv = abs(long_martin$cn-2)

In [10]:
by_chrom_cell = long_martin %>% group_by(chrom, name) %>% summarise(cnv_mean=mean(cnv))

`summarise()` has grouped output by 'chrom'. You can override using the
`.groups` argument.


In [28]:
master$name = master$`SampleID Martin`
master$srid = master$`SampleReadsID (tree ID)`
master$cellid = master$`Cell ID`
master$individual = master$`Individual Name`
master[,'Tissue.type.1'] = master[,'Tissue type 1']
master[,'Tissue.type.2'] = master[,'Tissue type 2'] 
master[,'Tissue.type.3'] = master[,'Tissue type 3'] 
master[,'Tissue.type.4'] = master[,'Tissue type 4']
master[,'Cell.type.1'] = master[,'Cell type 1']
master[,'Cell.type.2'] = master[,'Cell type 2']
master[,'Cell.type.3'] = master[,'Cell type 3']

In [31]:
cnv_data = by_chrom_cell %>% pivot_wider(names_from = chrom, values_from = cnv_mean) %>% left_join(master[,c("name", "srid", "cellid", "individual", "Tissue.type.1", 
"Tissue.type.2", "Tissue.type.3", "Tissue.type.4", "Cell.type.1", "Cell.type.2", "Cell.type.3")], by = "name")
cnv_data = cnv_data[cnv_data$srid != "#N/A",]
cnv_data = cnv_data[cnv_data$srid != "Not available",]
cnv_data = cnv_data[!is.na(cnv_data$srid),]
rownames(cnv_data) = cnv_data$srid

Warning message:
"Setting row names on a tibble is deprecated."


In [32]:
fwrite(cnv_data, "cnv_data.csv")

In [79]:
# yaara = read.csv("~/s/Ofir/cnv_csv/BC15-0267_mappingID_cnv_short_DETAIL44d.csv")
# yaara = read.csv("~/s/Ofir/cnv_csv/MM15-127_mappingID_cnv_short_DETAIL44e_snp12a_new_cells1.csv")
# yaara = read.csv("~/s/Ofir/cnv_csv/MM16-412_mappingID_cnv_short_DETAIL44e_snp12a.csv")
yaara = read.csv("~/s/Ofir/cnv_csv/MM16-423_mappingID_cnv_short_DETAIL44e_snp12a.csv")

# yaara = read.csv("~/s/Ofir/cnv_csv/795-09_mappingID_cnv_short_DETAIL44e_snp9b.csv")

In [80]:
inter_srs = intersect(as.character(yaara$Sample.Reads.ID), cnv_data$srid)
rownames(yaara) = as.character(yaara$Sample.Reads.ID)
rowf = rownames(yaara) %in% inter_srs
for(i in 1:22){
    yaara[,paste0("chrome_",i,"total_length_abbr")] = "NA"
    yaara[rowf,paste0("chrome_",i,"total_length_abbr")] = as.matrix(cnv_data)[inter_srs,as.character(i)]
}

In [81]:
# yaara[,c("Tissue.type.1", "Tissue.type.2", "Tissue.type.3", "Tissue.type.4", "Cell.type.1", "Cell.type.2", "Cell.type.3")] = "NA"
yaara[rowf, c("Tissue.type.1", "Tissue.type.2", "Tissue.type.3", "Tissue.type.4", "Cell.type.1", "Cell.type.2", "Cell.type.3")] = as.matrix(cnv_data)[inter_srs,c("Tissue.type.1", "Tissue.type.2", "Tissue.type.3", "Tissue.type.4", "Cell.type.1", "Cell.type.2", "Cell.type.3")]

In [82]:
yaara["41251",c("Tissue.type.1", "Tissue.type.2", "Tissue.type.3", "Tissue.type.4", "Cell.type.1", "Cell.type.2", "Cell.type.3")]

,Tissue.type.1,Tissue.type.2,Tissue.type.3,Tissue.type.4,Cell.type.1,Cell.type.2,Cell.type.3
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
41251,PT,NA,NA,NA,tumor,PT,


In [83]:
# fwrite(yaara, "~/s/Ofir/cnv_csv/BC15-0267_martin.csv")
# fwrite(yaara, "~/s/Ofir/cnv_csv/MM15-127_martin.csv")

# fwrite(yaara, "~/s/Ofir/cnv_csv/MM16-412_martin.csv")

fwrite(yaara, "~/s/Ofir/cnv_csv/MM16-423_martin.csv")
# fwrite(yaara, "~/s/Ofir/cnv_csv/795-09_martin.csv")

In [84]:
#conf_yaml_path <- "/home/dcsoft/s/yaara/configure yaml/configure BC14-2680 cnv NEW a4.yml"
#conf_yaml_path <- "/home/dcsoft/s/yaara/configure yaml/configure BC15-0267 cnv NEW a4.yml"
#conf_yaml_path <- "/home/dcsoft/s/yaara/configure yaml/configure BC16-0545cnv NEW a4.yml"
#conf_yaml_path <- "/home/dcsoft/s/yaara/configure yaml/configure 795-09 cnv NEW a4.yml"

# conf_yaml_path <- "/home/ofirr/s/Ofir/yamls/configure 795-09 cnv NEW a4.yml"
# conf_yaml_path <- "/home/ofirr/s/Ofir/yamls/configure BC15-0267 cnv NEW a4.yml"
# conf_yaml_path <- "/home/ofirr/s/Ofir/yamls/configure MM15-127 cnv NEW a4.yml"
# conf_yaml_path <- "/home/ofirr/s/Ofir/yamls/configure MM16-412 cnv NEW snps4.yml"
conf_yaml_path <- "/home/ofirr/s/Ofir/yamls/configure MM16-423 cnv NEW snps4.yml"

In [88]:
#frame parameters
#for 230
#width <- 80
#height <- 40

#for 465
# width <- 170*5
# height <- 60*3
width <- 297*1.5
height <- 210

#for 440
#width <- 100
#height <- 50

#for eye
#width <- 60
#height <-25

# for MM15-127
#width <- 430
#height <- 250

# for karolinska big
#width <- 9000
#height <- 200

#width <- 220
#height <- 120

In [89]:
id_tip_trim_flag <- TRUE
id_tip_prefix <- "ID"
id_tip_trim_start <- 3
id_tip_trim_end <-7
man_multiply_second_legend <- 0.54 #2.5
man_multiply_second_legend_text <-0.34
man_multiply_first_legend_text <-1 #6
man_multiply_first_legend_title_size <- 0.25
man_space_second_legend = 10
man_space_second_legend_multiplier=0.1
man_offset_for_highlight_legend_x=-6
a_4_output<- FALSE
man_adjust_image_of_second_legend <- 1.8

In [90]:
out <- func.print.lineage.tree(conf_yaml_path,simulate.p.value=FALSE,
                               height= height,width = width, 
                               man_adjust_image_of_second_legend=man_adjust_image_of_second_legend, man_adj_heat_loc = 0,
                               man_boot_x_offset= 0.025,man_adj_heat_loc2=3, man_adj_heat_loc3=6,
                               id_tip_trim_flag= id_tip_trim_flag, 
                               id_tip_trim_start= id_tip_trim_start,
                               id_tip_trim_end=id_tip_trim_end,
                               id_tip_prefix = id_tip_prefix,
                               debug_mode=FALSE,
                               debug_print_data_tree= FALSE,
                               man_multiply_second_legend= man_multiply_second_legend,
                               man_multiply_second_legend_text= man_multiply_second_legend_text,
                               man_adjust_elipse=0.25, #11.05
                               man_multiply_elipse=14.5,
                               man_multiply_first_legend_text= man_multiply_first_legend_text,
                               man_multiply_first_legend_title_size= man_multiply_first_legend_title_size,
                               man_space_second_legend=man_space_second_legend,
                               man_space_second_legend_multiplier=man_space_second_legend_multiplier,
                               man_offset_for_highlight_legend_x = man_offset_for_highlight_legend_x,
                               a_4_output= a_4_output)

Warning message in readLines(file, warn = readLines.warn):
"incomplete final line found on '/home/ofirr/s/Ofir/yamls/configure MM16-423 cnv NEW snps4.yml'"


[1] "Get mapping csv from: /home/dcsoft/s/Ofir/cnv_csv/MM16-423_martin.csv"
[1] "Tree number 1"
[1] "Get tree from: /home/dcsoft/s/trees/09_05_2023/MM16-423/output_tree_enriched_with_tbe_scores.newick"
[1] "Individual name is MM16-423"
[1] "disp_index is"
[1] 1
[1] "IN heat"
[1] "heat_list_len is"
[1] 3
[1] "att2 is"
[1] TRUE
[1] "In  check"
[1] "with_title is"
[1] "sigma_too_big"
[1] "with_value is"
[[1]]
[1] FALSE

[[2]]
[1] "FALSE"

[[3]]
[1] "F"

[[4]]
[1] "f"

[1] "col is"
  [1]  TRUE FALSE FALSE  TRUE FALSE  TRUE FALSE  TRUE  TRUE  TRUE FALSE  TRUE
 [13] FALSE  TRUE FALSE FALSE  TRUE FALSE  TRUE FALSE FALSE FALSE FALSE  TRUE
 [25] FALSE  TRUE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE FALSE FALSE  TRUE
 [37]  TRUE  TRUE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [49] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE
 [61]  TRUE  TRUE FALSE  TRUE  TRUE FALSE FALSE  TRUE FALSE  TRUE FALSE  TRUE
 [73] FALSE  TRUE FALSE FALSE  TRUE  TRUE FALSE F

Warning message in max(suppressWarnings(as.numeric(foc)), na.rm = TRUE):
"no non-missing arguments to max; returning -Inf"
Warning message in max(suppressWarnings(as.numeric(foc)), na.rm = TRUE):
"no non-missing arguments to max; returning -Inf"


[1] "man_boot_x_offset is"
[1] 0.025
[1] "text_mu is"
[1] 2


Scale for y is already present.
Adding another scale for y, which will replace the existing scale.


...

Warning message in names(guides)[to_change] <- paste0(names(guides), "_new"):
"number of items to replace is not a multiple of replacement length"


[1] "off is"
[1] 7.341063
[1] "length(colnames(dxdf440_for_heat[[j]])) is"
[1] 3
[1] "wi is"
[1] 0.1043478


Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.
Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.
Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.


[1] 3
[1] "off_base is"
[1] 0.6
[1] "param is"
[1] 3.810628
[1] "man_adj_heat_loc is"
[1] 0
[1] "off is"
[1] 13.41063
[1] "length(colnames(dxdf440_for_heat[[j]])) is"
[1] 1
[1] "wi is"
[1] 0.03478261


Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.
Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.
Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.
